**현재 세션(1개)의 모든 클릭 로그**를 대상으로 MLP, Gaussian Naive Bayes, Decision Tree, XGBoost, Logistic Regression, Linear SVM을 사용해서 구매 예측

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import itertools
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# <br>
# 전처리

In [2]:
온라인 = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv', encoding='utf-8')
온라인 = 온라인.sort_values(['clnt_id','sess_id','hit_seq']).reset_index(drop=True)
온라인.shape

/Users/yaelinjo/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3196362, 14)

In [3]:
# {clnt_id}_{sess_id}인 unique_id 생성
온라인['unique_id'] = list(map(lambda x,y: str(x)+'_'+str(y), 온라인.clnt_id, 온라인.sess_id))
온라인.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm,unique_id
0,1,1,1,0,A01,20190911,16:14,11880,NaN,과일선물세트,14.0,124.0,unknown,mobile_app,1_1
1,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,14.0,124.0,unknown,mobile_app,1_1
2,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,14.0,124.0,unknown,mobile_app,1_1
3,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,45.0,424.0,unknown,mobile_app,1_2
4,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,45.0,424.0,unknown,mobile_app,1_2


In [4]:
# 온라인 고객: 72,399명, 세션: 367,149개
온라인.clnt_id.nunique(), 온라인.unique_id.nunique()

(72399, 367149)

In [10]:
# 각 clnt_id별 session이 바뀌는 지점 index 저장
idx1 = 온라인.unique_id.drop_duplicates().index.tolist()
idx2 = idx1[1:] + [len(온라인)]

In [10]:
# session별 체류시간 = 다음 hit_pss_tm - 이전 hit_pss_tm
stay_tm = []
for i, j in tqdm_notebook(zip(idx1, idx2), total=len(idx1)):
    temp = 온라인.iloc[i:j].hit_pss_tm
    if len(temp) == 1:
        stay_tm.append(temp.tolist())
    else:
        temp = temp.diff().tolist()[1:]
        temp.append(temp[-1]) # 마지막은 이전 세션의 체류시간을 활용
        stay_tm.append(temp)
    #stay_tm = stay_tm + 온라인.iloc[i:j].hit_pss_tm.diff().fillna(0).tolist()

In [14]:
# list를 flatten
# stay_tm = sum(stay_tm, [])
온라인['time_length'] = list(itertools.chain.from_iterable(stay_tm))

In [16]:
# 밀리초 -> 초 변환
온라인['hit_pss_tm'] = np.ceil(온라인['hit_pss_tm']/1000)
온라인['time_length'] = np.ceil(온라인['time_length']/1000)

In [17]:
# 주말 및 공휴일 여부
holiday_diff = []
for i in 온라인.sess_dt.map(lambda x: str(x)[-3:]).tolist():
    if i in (['706','707','713','714','717','720','721','727','728',
               '803','804','810','811','815','817','818','824','825',
               '907','908','912','913','914','915','921','922','928','929']):
        holiday_diff.append(1)
    else:
        holiday_diff.append(0)
온라인['holiday_diff'] = holiday_diff

In [19]:
# keyword: session별 sech_kwd의 cumulative sum
온라인['sech_kwd'] = 온라인.sech_kwd.map(lambda x: 0 if str(x) == 'nan' else 1)
keyword = []
for i, j in tqdm_notebook(zip(idx1, idx2), total=len(idx1)):
    temp = 온라인.iloc[i:j].sech_kwd
    keyword.append(np.cumsum(temp.tolist()))
온라인['keyword'] = list(itertools.chain.from_iterable(keyword))

In [20]:
온라인['sess_dt'] = 온라인['sess_dt'].map(lambda x: str(x)[:-2]) # day제거하고 month(7,8,9월)만 남김
온라인['dvc_ctg_nm'].fillna('unknown',inplace=True) # device null값을 unknown으로 대체

In [21]:
# dummy variable 생성 - action_type, biz_unit, sess_dt, trfc_src, dvc_ctg_nm
온라인 = pd.concat([온라인, pd.get_dummies(온라인[['biz_unit','sess_dt','trfc_src','dvc_ctg_nm']], drop_first=True)], axis=1)
온라인 = pd.concat([온라인, pd.get_dummies(온라인.action_type, drop_first=True, prefix='action_type')], axis=1)
온라인 = 온라인.drop(['action_type','biz_unit','sess_dt','hit_tm','sech_kwd','tot_pag_view_ct', 'tot_sess_hr_v','trfc_src','dvc_ctg_nm'], axis=1)

In [23]:
온라인 = 온라인[['clnt_id', 'sess_id', 'trans_id', 'hit_seq', 'hit_pss_tm', 
           'time_length', 'holiday_diff', 'keyword', 'action_type_1', 'action_type_2',
           'action_type_3','action_type_4', 'action_type_5', 'action_type_6', 'action_type_7',
           'biz_unit_A02', 'biz_unit_A03', 'sess_dt_201908', 'sess_dt_201909', 'trfc_src_PORTAL_1',
           'trfc_src_PORTAL_2', 'trfc_src_PORTAL_3', 'trfc_src_PUSH', 'trfc_src_WEBSITE', 'trfc_src_unknown', 
           'dvc_ctg_nm_mobile_app','dvc_ctg_nm_mobile_web', 'dvc_ctg_nm_unknown', 'unique_id']]
print(온라인.shape)
온라인.head()

(3196362, 29)


,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_1,action_type_2,...,trfc_src_PORTAL_1,trfc_src_PORTAL_2,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id
0,1,1,NaN,1,12.0,11.0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,1_1
1,1,1,NaN,2,23.0,14.0,0,2,0,0,...,0,0,0,0,0,1,1,0,0,1_1
2,1,1,NaN,3,37.0,14.0,0,3,0,0,...,0,0,0,0,0,1,1,0,0,1_1
3,1,2,NaN,1,42.0,15.0,1,1,0,0,...,0,0,0,0,0,1,1,0,0,1_2
4,1,2,NaN,2,57.0,14.0,1,2,0,0,...,0,0,0,0,0,1,1,0,0,1_2


In [24]:
# 저장
온라인.to_csv('온라인_전처리_final.csv', index=False)

In [2]:
# 불러오기
import pandas as pd

온라인 = pd.read_csv('온라인_전처리_final.csv')

# <br>
# 종속변수 생성
다음 세션의 구매 여부

In [3]:
# 각 hit_seq 당 action_type이 구매완료=1, 이외=0
온라인['buy'] = 온라인['action_type_6']
온라인.drop('action_type_6', inplace=True, axis=1)
print(온라인.shape)
온라인.head()

(3196362, 29)


,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_1,action_type_2,...,trfc_src_PORTAL_2,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,0,0,...,0,0,0,0,1,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,0,0,...,0,0,0,0,1,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,0,0,...,0,0,0,0,1,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,0,0,...,0,0,0,0,1,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,0,0,...,0,0,0,0,1,1,0,0,1_2,0


In [4]:
#다음 세션의 구매를 예측하기 위한 종속변수 생성
구매여부 = 온라인[['clnt_id', 'sess_id', 'buy']].groupby(['clnt_id', 'sess_id']).sum()
구매여부.buy = 구매여부.buy.apply(lambda x:0 if x == 0 else 1)
구매여부 = 구매여부.sort_index()
구매여부 = 구매여부.reset_index()
#각 clnt_id별로 shift로 1행씩 올림
구매여부.buy = 구매여부.buy.shift(-1)
g = 구매여부.groupby('clnt_id')
#1행씩 올리면 각 clnt_id별로 마지막 값은 다음 id의 값, 따라서 마지막 행 제거
구매여부.drop(g.tail(1).index, axis=0, inplace = True)

In [5]:
구매여부

,clnt_id,sess_id,buy
0,1,1,0.0
2,2,1,0.0
3,2,2,0.0
4,2,3,0.0
5,2,4,0.0
...,...,...,...
367141,72427,3,0.0
367143,72428,1,0.0
367144,72428,2,0.0
367145,72428,3,0.0


In [6]:
온라인.drop(['buy'], axis =1, inplace= True)
온라인 = pd.merge(온라인, 구매여부, left_on=['clnt_id', 'sess_id'], right_on=['clnt_id', 'sess_id'])

In [7]:
print(온라인.shape)
온라인.head()

(2750164, 29)


,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_1,action_type_2,...,trfc_src_PORTAL_2,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,0,0,...,0,0,0,0,1,1,0,0,1_1,0.0
1,1,1,NaN,2,23.0,14.0,0,2,0,0,...,0,0,0,0,1,1,0,0,1_1,0.0
2,1,1,NaN,3,37.0,14.0,0,3,0,0,...,0,0,0,0,1,1,0,0,1_1,0.0
3,2,1,NaN,1,40.0,8.0,1,1,0,0,...,0,0,0,0,0,0,0,1,2_1,0.0
4,2,1,NaN,2,48.0,1.0,1,1,1,0,...,0,0,0,0,0,0,0,1,2_1,0.0


# <br>
# [1-2]현재 세션(1개)의 모든 클릭 로그 사용
(session, sequence, variables) 3d array를 1d array로 변환하여 활용

In [22]:
from keras.preprocessing import sequence
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split

In [11]:
# (session, sequence, variables) 3d array 변환
온라인_x = []
for i, j in tqdm_notebook(zip(idx1, idx2), total=len(idx1)):
    온라인_x.append(온라인.iloc[i:j, 3:-2].values)
    
#scaler = StandardScaler() 
#scaler = MinMaxScaler(feature_range=(0, 1))
#온라인_x = scaler.fit_transform(np.array(온라인_x))
온라인_x = np.array(온라인_x)

In [12]:
# session 당 구매 여부
온라인_y = []
for i,j in tqdm_notebook(zip(idx1,idx2), total=len(idx1)):
    온라인_y.append([int(온라인.buy.iloc[i:j].sum()>0)])

In [13]:
len(온라인_y)

294750

In [18]:
def make_padding_and_oversample(X, Y, length=70):
    max_len = length
    X_padding = sequence.pad_sequences(X, maxlen=max_len, padding='pre', truncating='post')
    X_padding2 = X_padding.reshape(X.shape[0], max_len* X_padding.shape[2])

    smote = SMOTE(random_state=0)
    X_resampled, Y_resampled = smote.fit_resample(X_padding2, Y)
    X_resampled = X_resampled.reshape(X_resampled.shape[0], max_len, X_padding.shape[2])
    return X_padding, X_resampled, Y_resampled

In [19]:
idx = list(pd.Series(idx2) - pd.Series(idx1))
length = max(idx)

In [24]:
X_padded, X_resampled, Y_resampled = make_padding_and_oversample(온라인_x, 온라인_y, length=length)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, Y_resampled, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

(361313, 357, 24)
(154849, 357, 24)


In [29]:
num_train = X_train.shape[0]
num_test = X_test.shape[0]

In [30]:
X_train_1d = X_train.reshape((num_train,-1))
X_test_1d = X_test.reshape((num_test,-1))

# <br>
# 모델링

In [80]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.layers import Dropout

import pickle
from joblib import dump, load

In [45]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Gaussian Naive Bayes

In [34]:
clf = GaussianNB()
clf.fit(X_train_1d, y_train)
y_pred = clf.predict(X_test_1d)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.52
F1 score: 0.67
Precision: 0.51
Recall: 0.98


In [76]:
dump(clf, '1-2-Gaussian.joblib')

['1-2-Gaussian.joblib']

### Decision Tree

In [36]:
clf2 = DecisionTreeClassifier().fit(X_train_1d, y_train)
y_pred = clf2.predict(X_test_1d)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.82
F1 score: 0.82
Precision: 0.81
Recall: 0.82


In [77]:
dump(clf2, '1-2-DecisionTree.joblib')

['1-2-DecisionTree.joblib']

### XGBOOST

In [ ]:
clf3

### Logistic Regression

In [40]:
# clf4 = LogisticRegression().fit(X_train_1d, y_train)
y_pred = clf4.predict(X_test_1d)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.58
F1 score: 0.67
Precision: 0.55
Recall: 0.84


In [78]:
dump(clf4, '1-2-LogisticRegression.joblib')

['1-2-LogisticRegression.joblib']

### Linear SVM

In [43]:
clf5 = LinearSVC().fit(X_train_1d, y_train)
y_pred = clf5.predict(X_test_1d)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

/Users/yaelinjo/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.71
F1 score: 0.73
Precision: 0.68
Recall: 0.79


In [79]:
dump(clf4, '1-2-LinearSVM.joblib')

['1-2-LinearSVM.joblib']

### MLP(DNN)

In [51]:
X_train_1d.shape

(361313, 8568)

In [86]:
def models(train):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(train.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr= 0.001, rho = 0.9), metrics=['acc', f1_m, precision_m, recall_m])
    return model

In [87]:
model = models(X_train_1d)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 32)                274208    
_________________________________________________________________
dense_18 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 17        
Total params: 274,753
Trainable params: 274,753
Non-trainable params: 0
_________________________________________________________________


In [88]:
history = model.fit(X_train_1d, y_train, epochs=25, batch_size=1000, validation_data=(X_test_1d, y_test), verbose=2, shuffle=True)

Train on 361313 samples, validate on 154849 samples
Epoch 1/25
 - 40s - loss: 1.1047 - acc: 0.6469 - f1_m: 0.5812 - precision_m: 0.7110 - recall_m: 0.5451 - val_loss: 0.6927 - val_acc: 0.7027 - val_f1_m: 0.7199 - val_precision_m: 0.6807 - val_recall_m: 0.7643
Epoch 2/25
 - 37s - loss: 0.6639 - acc: 0.7287 - f1_m: 0.7088 - precision_m: 0.7740 - recall_m: 0.6846 - val_loss: 0.5280 - val_acc: 0.7863 - val_f1_m: 0.7542 - val_precision_m: 0.8874 - val_recall_m: 0.6561
Epoch 3/25
 - 38s - loss: 0.5307 - acc: 0.7569 - f1_m: 0.7445 - precision_m: 0.7868 - recall_m: 0.7279 - val_loss: 0.4728 - val_acc: 0.7733 - val_f1_m: 0.7821 - val_precision_m: 0.7527 - val_recall_m: 0.8143
Epoch 4/25
 - 37s - loss: 0.5000 - acc: 0.7695 - f1_m: 0.7560 - precision_m: 0.8056 - recall_m: 0.7321 - val_loss: 0.4812 - val_acc: 0.7925 - val_f1_m: 0.7769 - val_precision_m: 0.8399 - val_recall_m: 0.7231
Epoch 5/25
 - 38s - loss: 0.4785 - acc: 0.7781 - f1_m: 0.7636 - precision_m: 0.8177 - recall_m: 0.7329 - val_loss: 0

In [89]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test_1d, y_test, verbose=1)

154849/154849 [==============================] - 7s 48us/step


In [90]:
print(loss)
print(accuracy)
print(f1_score)
print(precision)
print(recall)

0.3751236852681496
0.8331923357593527
0.819938173571149
0.8687277610175301
0.7854875094380235


In [91]:
model.save('1-2-DNN.h5')